In [42]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import Xception
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from sklearn.model_selection import train_test_split 
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score
import torch
from torchvision.ops.boxes import _box_inter_union
import tensorflow_addons as tfa
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)
from tensorflow import keras

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [45]:
normal_path = '/content/gdrive/MyDrive/Projects/Machine Learning for Healthcare/dataset/Non Cancerous'
carcinoma_path = '/content/gdrive/MyDrive/Projects/Machine Learning for Healthcare/dataset/Cancerous'

In [46]:
# creating an empty list, iterating over each file in image folder, converting them into array and appending them in the created list
normal_images = []

for filename in sorted(os.listdir(normal_path)):
  normal_image_fullpath = os.path.join(normal_path, filename)
  normal_img = load_img(normal_image_fullpath, target_size = (224, 224))           
  normal_img_arr = img_to_array(normal_img)
  normal_images.append(normal_img_arr)

# converting list into numpy arrays
normal_images = np.array(normal_images)

In [47]:
normal_images.shape

(290, 224, 224, 3)

In [48]:
# creating an empty list, iterating over each file in image folder, converting them into array and appending them in the created list
carcinoma_images = []

for filename in sorted(os.listdir(carcinoma_path)):
  carcinoma_image_fullpath = os.path.join(carcinoma_path, filename)
  carcinoma_img = load_img(carcinoma_image_fullpath, target_size = (224, 224)) 
  carcinoma_img_arr = img_to_array(carcinoma_img)
  carcinoma_images.append(carcinoma_img_arr)

# converting list into numpy arrays
carcinoma_images = np.array(carcinoma_images)

In [49]:
carcinoma_images.shape

(934, 224, 224, 3)

In [50]:
# joining normal and carcinoma dataset
X = np.concatenate((normal_images, carcinoma_images), axis = 0)
X.shape

(1224, 224, 224, 3)

In [51]:
# creating labels
normal_labels = np.zeros((normal_images.shape[0], 1))
carcinoma_labels = np.ones((carcinoma_images.shape[0], 1))
y = np.concatenate((normal_labels, carcinoma_labels), axis = 0)
y.shape

(1224, 1)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size = 0.2)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, shuffle = True, test_size = 0.33)

In [53]:
print("shape of X_Train = ", X_train.shape)
print("shape of X_Test = ", X_test.shape)
print("shape of X_val = ", X_val.shape)
print("shape of y_train = ", y_train.shape)
print("shape of y_test = ", y_test.shape)
print("shape of y_val = ", y_val.shape)

shape of X_Train =  (979, 224, 224, 3)
shape of X_Test =  (81, 224, 224, 3)
shape of X_val =  (164, 224, 224, 3)
shape of y_train =  (979, 1)
shape of y_test =  (81, 1)
shape of y_val =  (164, 1)


In [54]:
image_size = [224, 224]

# loading the pretrained vgg19 model(pretrained on imagenet dataset) from keras
vgg16 = VGG16(input_shape = image_size+[3], weights = 'imagenet', include_top = False)

# freezing the top layers in the model
for layer in vgg16.layers:
  layer.trainable = False

# building the custom layers for bb regression on top of vgg19 model
out = vgg16.output
flat = Flatten()(out)
FCL1 = Dense(128, activation = 'relu')(flat)
FCL2 = Dense(32, activation = 'relu')(FCL1)
OutR = Dense(1, activation = 'sigmoid')(FCL2)

v_model = Model(inputs = vgg16.input, outputs = OutR)

# using mean squared error loss and general intersection over union metric for the model
v_model.compile(
    loss = tensorflow.keras.losses.BinaryCrossentropy(),
    optimizer = 'adam',
    metrics = 'accuracy'
)
v_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [55]:
v_trained_model = v_model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                batch_size = 10,
                epochs = 32,
                shuffle = True,
                verbose = 1)

Epoch 1/32
98/98 [==============================] - 9s 85ms/step - loss: 3.4381 - accuracy: 0.7273 - val_loss: 1.3570 - val_accuracy: 0.7500
Epoch 2/32
98/98 [==============================] - 7s 73ms/step - loss: 0.7301 - accuracy: 0.8836 - val_loss: 0.7467 - val_accuracy: 0.8476
Epoch 3/32
98/98 [==============================] - 7s 76ms/step - loss: 0.3467 - accuracy: 0.9213 - val_loss: 1.2416 - val_accuracy: 0.8537
Epoch 4/32
98/98 [==============================] - 7s 75ms/step - loss: 0.2504 - accuracy: 0.9540 - val_loss: 0.9905 - val_accuracy: 0.7866
Epoch 5/32
98/98 [==============================] - 7s 75ms/step - loss: 0.1715 - accuracy: 0.9745 - val_loss: 0.8464 - val_accuracy: 0.8598
Epoch 6/32
98/98 [==============================] - 7s 76ms/step - loss: 0.1605 - accuracy: 0.9642 - val_loss: 0.6219 - val_accuracy: 0.8902
Epoch 7/32
98/98 [==============================] - 7s 76ms/step - loss: 0.1396 - accuracy: 0.9765 - val_loss: 0.8348 - val_accuracy: 0.8049
Epoch 8/32
98

In [56]:
ypred_v = v_model.predict(X_test)
print(accuracy_score(y_test, np.round(ypred_v)))
print(precision_score(y_test, np.round(ypred_v)))
print(recall_score(y_test, np.round(ypred_v)))
print(f1_score(y_test, np.round(ypred_v)))

3/3 [==============================] - 2s 1s/step
0.8271604938271605
0.9193548387096774
0.8636363636363636
0.890625


In [57]:
image_size = [224, 224]

# loading the pretrained resnet50 model(pretrained on imagenet dataset) from keras
resnet = ResNet50(input_shape = image_size+[3], weights = 'imagenet', include_top = False)

# freezing the top layers in the model
for layer in resnet.layers:
  layer.trainable = False

# building the custom layers for bb regression on top of vgg19 model
out = resnet.output
flat = Flatten()(out)
FCL1 = Dense(128, activation = 'relu')(flat)
FCL2 = Dense(32, activation = 'relu')(FCL1)
OutR = Dense(1, activation = 'sigmoid')(FCL2)

r_model = Model(inputs = resnet.input, outputs = OutR)

# using mean squared error loss and general intersection over union metric for the model
r_model.compile(
    loss = tensorflow.keras.losses.BinaryCrossentropy(),
    optimizer = 'adam',
    metrics = 'accuracy'
)
r_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [58]:
r_trained_model = r_model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                batch_size = 10,
                epochs = 32,
                shuffle = True,
                verbose = 1)

Epoch 1/32
98/98 [==============================] - 9s 59ms/step - loss: 2.6971 - accuracy: 0.7395 - val_loss: 1.0486 - val_accuracy: 0.7744
Epoch 2/32
98/98 [==============================] - 4s 45ms/step - loss: 0.6422 - accuracy: 0.9254 - val_loss: 1.2097 - val_accuracy: 0.7805
Epoch 3/32
98/98 [==============================] - 4s 45ms/step - loss: 0.4231 - accuracy: 0.9316 - val_loss: 1.0130 - val_accuracy: 0.8841
Epoch 4/32
98/98 [==============================] - 4s 45ms/step - loss: 0.3911 - accuracy: 0.9602 - val_loss: 0.6314 - val_accuracy: 0.8537
Epoch 5/32
98/98 [==============================] - 4s 45ms/step - loss: 0.3079 - accuracy: 0.9581 - val_loss: 1.0696 - val_accuracy: 0.8049
Epoch 6/32
98/98 [==============================] - 4s 45ms/step - loss: 0.1153 - accuracy: 0.9745 - val_loss: 0.7032 - val_accuracy: 0.8780
Epoch 7/32
98/98 [==============================] - 4s 46ms/step - loss: 0.1595 - accuracy: 0.9755 - val_loss: 1.1338 - val_accuracy: 0.7561
Epoch 8/32
98

In [59]:
ypred_r = r_model.predict(X_test)
print(accuracy_score(y_test, np.round(ypred_r)))
print(precision_score(y_test, np.round(ypred_r)))
print(recall_score(y_test, np.round(ypred_r)))
print(f1_score(y_test, np.round(ypred_r)))

3/3 [==============================] - 2s 392ms/step
0.8024691358024691
0.8571428571428571
0.9090909090909091
0.8823529411764706


In [60]:
image_size = [224, 224]

# loading the pretrained inceptionv3 model(pretrained on imagenet dataset) from keras
inception = InceptionV3(input_shape = image_size+[3], weights = 'imagenet', include_top = False)

# freezing the top layers in the model
for layer in inception.layers:
   layer.trainable = False

# building the custom layers for bb regression on top of vgg19 model
out = inception.output
flat = Flatten()(out)
FCL1 = Dense(128,activation = 'relu')(flat)
FCL2 = Dense(32,activation = 'relu')(FCL1)
OutR = Dense(1, activation = 'sigmoid')(FCL2)

i_model = Model(inputs = inception.input, outputs = OutR)

# using mean squared error loss and general intersection over union metric for the model
i_model.compile(
    loss = tensorflow.keras.losses.BinaryCrossentropy(),
    optimizer = 'adam',
    metrics = 'accuracy'
)
i_model.summary()

87910968/87910968 [==============================] - 1s 0us/step
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                

In [61]:
i_trained_model = i_model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                batch_size = 10,
                epochs = 32,
                shuffle = True,
                verbose = 1)

Epoch 1/32
98/98 [==============================] - 12s 63ms/step - loss: 7.3974 - accuracy: 0.6680 - val_loss: 1.9451 - val_accuracy: 0.6220
Epoch 2/32
98/98 [==============================] - 3s 35ms/step - loss: 2.3368 - accuracy: 0.7385 - val_loss: 1.6271 - val_accuracy: 0.7500
Epoch 3/32
98/98 [==============================] - 3s 35ms/step - loss: 2.3639 - accuracy: 0.7518 - val_loss: 1.7613 - val_accuracy: 0.8110
Epoch 4/32
98/98 [==============================] - 3s 35ms/step - loss: 1.2819 - accuracy: 0.7835 - val_loss: 1.9886 - val_accuracy: 0.8232
Epoch 5/32
98/98 [==============================] - 3s 35ms/step - loss: 0.7307 - accuracy: 0.8386 - val_loss: 1.8858 - val_accuracy: 0.5610
Epoch 6/32
98/98 [==============================] - 3s 36ms/step - loss: 0.3749 - accuracy: 0.8754 - val_loss: 1.1652 - val_accuracy: 0.7988
Epoch 7/32
98/98 [==============================] - 3s 36ms/step - loss: 0.4262 - accuracy: 0.8815 - val_loss: 2.2328 - val_accuracy: 0.4878
Epoch 8/32
9

In [62]:
ypred_i = i_model.predict(X_test)
print(accuracy_score(y_test, np.round(ypred_i)))
print(precision_score(y_test, np.round(ypred_i)))
print(recall_score(y_test, np.round(ypred_i)))
print(f1_score(y_test, np.round(ypred_i)))

3/3 [==============================] - 5s 525ms/step
0.7160493827160493
0.864406779661017
0.7727272727272727
0.8159999999999998


In [63]:
image_size = [224, 224, 3]

con_inputs = keras.Input(shape=image_size, name = "img")

# loading the pretrained vgg19 model (pretrained on imagenet dataset) from keras
vgg = VGG16(input_tensor = con_inputs, input_shape=image_size, weights = 'imagenet', include_top = False)

for layer in vgg.layers:
  layer.trainable = False

flat_vgg = keras.layers.GlobalAveragePooling2D()(vgg.output)

# loading the pretrained resnet50 model (pretrained on imagenet dataset) from keras
resnet = ResNet50(input_tensor = con_inputs,input_shape=image_size, weights = 'imagenet', include_top = False)

for layer in resnet.layers:
  layer.trainable = False

flat_resnet = keras.layers.GlobalAveragePooling2D()(resnet.output)

# loading the pretrained inceptionv3 model (pretrained on imagenet dataset) from keras
inception = InceptionV3(input_tensor=con_inputs,input_shape=image_size, weights = 'imagenet', include_top = False)

# freezing the top layers in the model
for layer in inception.layers:
  layer.trainable = False

flat_inception = keras.layers.GlobalAveragePooling2D()(inception.output)

flat = keras.layers.Concatenate()([flat_vgg, flat_resnet, flat_inception])

first = Dense(1024, activation = 'relu')(flat)
prev = Dense(512, activation = 'relu')(first)
FCL1 = Dense(128, activation = 'relu')(prev)
FCL2 = Dense(32, activation = 'relu')(FCL1)
OutR = Dense(1, activation = 'sigmoid')(FCL2)


c2_model = Model(inputs = con_inputs, outputs = OutR)

# using mean squared error loss and general intersection over union metric for the model
c2_model.compile(
    loss = tensorflow.keras.losses.BinaryCrossentropy(),
    optimizer = 'adam',
    metrics = 'accuracy'
)
c2_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['img[0][0]']                    
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [64]:
c2_trained_model = c2_model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                batch_size = 10,
                epochs = 32,
                shuffle = True,
                verbose = 1)

Epoch 1/32
98/98 [==============================] - 28s 179ms/step - loss: 2.5290 - accuracy: 0.6915 - val_loss: 0.4964 - val_accuracy: 0.8049
Epoch 2/32
98/98 [==============================] - 13s 137ms/step - loss: 0.6686 - accuracy: 0.7436 - val_loss: 0.4810 - val_accuracy: 0.8110
Epoch 3/32
98/98 [==============================] - 13s 137ms/step - loss: 0.4903 - accuracy: 0.7773 - val_loss: 0.9932 - val_accuracy: 0.4512
Epoch 4/32
98/98 [==============================] - 13s 136ms/step - loss: 0.4386 - accuracy: 0.7978 - val_loss: 0.3429 - val_accuracy: 0.8537
Epoch 5/32
98/98 [==============================] - 13s 134ms/step - loss: 0.3698 - accuracy: 0.8182 - val_loss: 0.4501 - val_accuracy: 0.8537
Epoch 6/32
98/98 [==============================] - 13s 133ms/step - loss: 0.3223 - accuracy: 0.8601 - val_loss: 0.4647 - val_accuracy: 0.8049
Epoch 7/32
98/98 [==============================] - 13s 135ms/step - loss: 0.3138 - accuracy: 0.8672 - val_loss: 0.3594 - val_accuracy: 0.8598

In [65]:
ypred_c2 = c2_model.predict(X_test)
print(accuracy_score(y_test, np.round(ypred_c2)))
print(precision_score(y_test, np.round(ypred_c2)))
print(recall_score(y_test, np.round(ypred_c2)))
print(f1_score(y_test, np.round(ypred_c2)))

3/3 [==============================] - 7s 1s/step
0.8518518518518519
0.8857142857142857
0.9393939393939394
0.9117647058823529
